In [2]:
#Importando librerias

import pandas as pd
import sqlite3 as sql


In [3]:
conn = sql.connect('sql-murder-mystery.db') 
def tables_in_sqlite_db(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tables
tables = tables_in_sqlite_db(conn)
print(tables)

#SELECT name FROM sqlite_master WHERE type='table'




['crime_scene_report', 'drivers_license', 'person', 'facebook_event_checkin', 'interview', 'get_fit_now_member', 'get_fit_now_check_in', 'income', 'solution']


In [69]:

query = '''SELECT 
    A.date,
    A.type,
    A.city,
    A.description
    
FROM crime_scene_report A
WHERE A.date =20180115 AND type = 'murder'


'''
data = pd.read_sql(query, conn) 
data
#aqui podemos ver que el 15 de enero del 2018 solo hubo 1 asesinato en SQL city y la descripcion dice: 
#Security footage shows that there were 2 witnesses.
#The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, 
#lives somewhere on "Franklin Ave"

,date,type,city,description
0,20180115,murder,Albany,Life? Dont talk to me about life.
1,20180115,murder,Reno,"Mama, I killed a man, put a gun against his he..."
2,20180115,murder,SQL City,Security footage shows that there were 2 witne...


In [41]:
query = '''SELECT id,name,license_id,MAX(address_number), address_street_name,ssn FROM person WHERE address_street_name = 'Northwestern Dr' 
    

'''
data = pd.read_sql(query, conn) 
data

#Aqui tenemos al primer testigo que vive en la ultima casa de Northwestern DR

,id,name,license_id,MAX(address_number),address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [59]:

query2 = '''SELECT * FROM person WHERE address_street_name = 'Franklin Ave' AND name LIKE 'Annabel%'
    

'''
data2 = pd.read_sql(query2, conn) 
data2

    
# Segundo testigo que se llama Annabel y vive en algun lugar de Franklin Ave

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [56]:
query = '''
SELECT * FROM drivers_license 
WHERE plate_number LIKE 'H42W%' 

'''
data = pd.read_sql(query, conn) 
data

#AND(id = 28819 OR id = 67318)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius


In [47]:
query = '''
SELECT * FROM facebook_event_checkin
WHERE person_id = 14887
OR person_id = 16371 

'''
#16371 annabel
# 14887 Morty
data = pd.read_sql(query, conn) 
data

#Esta informacion nos sirve porque podemos corroborar que ambas personas vieron al asesino porque fueron al mismo evento

,person_id,event_id,event_name,date
0,14887,4719,The Funky Grooves Tour,20180115
1,16371,4719,The Funky Grooves Tour,20180115


In [48]:
query = '''
SELECT * FROM interview
WHERE person_id = 14887
OR person_id = 16371 
'''
data = pd.read_sql(query, conn) 
data

#DECLARACION DE MORTY
#I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag.
#The membership number on the bag started with "48Z". 
#Only gold members have those bags. The man got into a car with a plate that included "H42W"
################################
#DECLARACION DE ANNABELE
#I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


#aqui ya sabemos que el ladron va al gimnasio con una membresia gold que comienza con 48Z y las placas del carro incluyen "H42W"
#tambien ya sabemos que vieron al asesino en el gimnasio el nueve de enero, eso nos confirma que efectivamente es miembro

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


In [50]:
query = '''
SELECT * FROM get_fit_now_member
WHERE membership_status = 'gold'
AND id LIKE '48Z%'
'''
data = pd.read_sql(query, conn) 
data

#aqui tenemos a dos sospechosos del asesinato, ambos tienen membresia gold y su ID comiena con 48Z
#Joe Germuska person id = 28819
# Jeremy Bowers person id = 67318

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [54]:
query = '''
SELECT * FROM get_fit_now_check_in
WHERE check_in_date = 20180109 AND membership_id LIKE '48Z%'
'''
data = pd.read_sql(query, conn) 
data

#Aqui se complica un poco porque ambos fueron al gimnasio el mismo día, solo la hora de salida es similar

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [62]:
query = '''
SELECT * FROM income
'''
data = pd.read_sql(query, conn) 
data

,ssn,annual_income
0,100009868,52200
1,100169584,64500
2,100300433,74400
3,100355733,35900
4,100366269,73000
...,...,...
7509,999679296,54400
7510,999762859,77000
7511,999824984,82000
7512,999910617,82600


In [74]:
query = '''
SELECT * FROM solution
'''
data = pd.read_sql(query, conn) 
data



,user,value
0,0,"Congrats, you found the brains behind the murd..."


In [65]:
cursor = conn.cursor() 
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql) 
conn.commit()
var = pd.read_sql("SELECT value FROM solution", conn) 
var.iloc[0,0]

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [135]:
query = '''
SELECT * FROM interview
WHERE person_id =  67318
'''
data = pd.read_sql(query, conn) 
data

#Tenemos la declaracion del asesino 


#I was hired by a woman with a lot of money. 
#I don't know her name but I know she's around 5'5" (65") or 5'7" (67").
#She has red hair and she drives a Tesla Model S. 
#I know that she attended the SQL Symphony Concert 3 times in December 2017.

,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


In [144]:
query = '''
SELECT * FROM drivers_license 
WHERE height BETWEEN 65 AND 67 AND gender = 'female' AND hair_color = 'red' 
AND car_make = 'Tesla'


'''
data = pd.read_sql(query, conn) 
data

#id sospechosos : 
    #202298
    #291182
    #918773

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [154]:
query = '''
SELECT * FROM person
WHERE license_id = 202298 OR license_id = 291182 OR license_id = 918773
'''

data = pd.read_sql(query, conn) 
data

#3 sospechosas

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910
1,90700,Regina George,291182,332,Maple Ave,337169072
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [158]:
query = '''
SELECT * FROM facebook_event_checkin
WHERE event_name ='SQL Symphony Concert' AND person_id = 78881 
OR person_id = 90700 OR person_id = 99716
'''

data = pd.read_sql(query, conn) 
data

#Aqui confirmamos que Miranda Priestly fue 3 veces en el 2017


,person_id,event_id,event_name,date
0,99716,1143,SQL Symphony Concert,20171206
1,99716,1143,SQL Symphony Concert,20171212
2,99716,1143,SQL Symphony Concert,20171229


In [159]:
cursor = conn.cursor() 
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')"
cursor.execute(sql) 
conn.commit()
var = pd.read_sql("SELECT value FROM solution", conn) 
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'